### TRAINING ONLY ON COLAB DRIVE

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -r gdrive/MyDrive/szlaban/yolov7/requirements.txt

In [ ]:
!cat gdrive/MyDrive/szlaban/yolov7/plate.yaml

train: ../plate_training/dataset/images
val: ../plate_training/dataset/images

nc: 1
names: ['plate']

Copy data from drive to colab to awoid git tag error

In [4]:
!cp -r gdrive/MyDrive/szlaban ./szlaban

Check directory tree

In [7]:
!ls ./szlaban/plate_training/dataset

annotations	     data_aug.py  scripts     xml_annotations_big
annotations_txt.zip  images	  source.txt


In [10]:
!nano ./szlaban/yolov7/plate.yaml

/bin/bash: nano: command not found


In [11]:
'''
Delete old data
'''
import os
import shutil

images = os.listdir('./szlaban/plate_training/dataset/images')
labels = os.listdir('./szlaban/plate_training/dataset/labels')


for image in images:
  if '_' in image:
    os.remove(f'./szlaban/plate_training/dataset/images/{image}')

for label in labels:
  if '_' in label:
    os.remove(f'./szlaban/plate_training/dataset/labels/{label}')

In [12]:
'''
Labels and images test
'''

import os

num_images = os.listdir('./szlaban/plate_training/dataset/images')
num_labels = os.listdir('./szlaban/plate_training/dataset/labels')

print(len(num_images))
print(len(num_labels))

433
433


In [13]:
'''
Fix labels
'''
labels = os.listdir('./szlaban/plate_training/dataset/labels')
labels_dir = './szlaban/plate_training/dataset/labels'


for label in labels:
  file = open(f'{labels_dir}/{label}')
  file_content = file.read()
  changed_file_content = file_content.replace('licence', '0')
  os.remove(f'{labels_dir}/{label}')
  new_file = open(f'{labels_dir}/{label}', 'a')
  new_file.write(changed_file_content)
  new_file.close() 


PoC

In [ ]:
!python szlaban/yolov7/train.py --batch-size 32 --device 0 --cache --data szlaban/yolov7/plate.yaml --img 640 640 --cfg szlaban/yolov7/cfg/training/yolov7.yaml --weights szlaban/yolov7/yolov7_training.pt --name yolov7-custom --hyp szlaban/yolov7/data/hyp.scratch.custom.yaml

In [ ]:
!zip -r ./gdrive/MyDrive/szlaban_poc.zip ./szlaban

In [60]:
!cp -R szlaban ./gdrive/MyDrive/szlaban/poc/

In [66]:
!ls gdrive/MyDrive/szlaban/poc/szlaban/runs -ltr

total 4
drwx------ 5 root root 4096 Oct 17 14:33 detect


In [51]:
!ls runs/train/yolov7-custom15/weights

best_200.pt  best_233.pt  best.pt	epoch_174.pt  epoch_298.pt
best_204.pt  best_234.pt  epoch_000.pt	epoch_199.pt  epoch_299.pt
best_205.pt  best_235.pt  epoch_024.pt	epoch_224.pt  init.pt
best_207.pt  best_258.pt  epoch_049.pt	epoch_249.pt  last.pt
best_209.pt  best_265.pt  epoch_074.pt	epoch_274.pt
best_217.pt  best_270.pt  epoch_099.pt	epoch_295.pt
best_220.pt  best_273.pt  epoch_124.pt	epoch_296.pt
best_227.pt  best_280.pt  epoch_149.pt	epoch_297.pt


In [69]:
!cp -r runs ./gdrive/MyDrive/szlaban/poc/poc_train_run

In [52]:
!cp ./gdrive/MyDrive/szlaban/inference/sportage.jpg ./inference/sportage.jpg

In [55]:
import cv2 as cv

image = cv.imread('./inference/sportage.jpg')
image2 = cv.resize(image, (640, 640))

cv.imwrite('./inference/sportage_resized.jpg', image2)


True

In [57]:
!python szlaban/yolov7/detect.py --weights runs/train/yolov7-custom15/weights/best.pt --conf 0.25 --img-size 640 --source inference/sportage_resized.jpg

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='inference/sportage_resized.jpg', update=False, view_img=False, weights=['runs/train/yolov7-custom15/weights/best.pt'])
YOLOR 🚀 2022-10-17 torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-de